In [0]:
%sql
-- # 1. Find out which airports when treated as Origin have the smallest and the largest departure delays.
CREATE OR REPLACE TEMP VIEW origin_delay_stats AS
SELECT
  Origin,
  AVG(DepDelay) AS avg_dep_delay,
  MIN(DepDelay) AS min_dep_delay,
  MAX(DepDelay) AS max_dep_delay
FROM aero2.flights
WHERE DepDelay IS NOT NULL
GROUP BY Origin;

In [0]:
%sql
(SELECT
  'MIN' AS TYPE,
  s.Origin,
  a.AIRPORT AS Origin_Airport,
  s.min_dep_delay AS Delay
FROM origin_delay_stats s
JOIN workspace.aero2.airports a
  ON s.Origin = a.IATA_CODE
ORDER BY s.min_dep_delay ASC
LIMIT 1)

UNION ALL

(SELECT
  'MAX' AS TYPE,
  s.Origin,
  a.AIRPORT AS Origin_Airport,
  s.max_dep_delay AS Delay
FROM origin_delay_stats s
JOIN workspace.aero2.airports a
  ON s.Origin = a.IATA_CODE
ORDER BY s.max_dep_delay DESC
LIMIT 1);

In [0]:
%sql
-- # 2. Create a list of U.S. States with the number of airports that each of them has in descending order. Show this on a map of the U.S. (Hint: Use: a choropleth map. States are listed as USPS Abbreviations)

CREATE OR REPLACE TEMP VIEW state_airport_counts AS
SELECT
STATE AS State,
  COUNT(*) AS Num_Airports
FROM workspace.aero2.airports
WHERE STATE IS NOT NULL
GROUP BY STATE
ORDER BY Num_Airports DESC;

In [0]:
%sql
SELECT * FROM state_airport_counts;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- # 3. Create a list containing: DATE, NUM_OF_FLIGHTS, MAX_ARRIVAL_DELAY, AVG_ARRIVAL_DELAY. Please provide the date (from FlightDate) in the following format i.e. 2018-01-10. The maximum and Average delays should be provided in minutes but rounded to 2 digits after the decimal point.

CREATE OR REPLACE TEMP VIEW daily_flight_stats AS
SELECT
  DATE_FORMAT(FlightDate, 'yyyy-MM-dd') AS DATE,
  COUNT(*) AS NUM_OF_FLIGHTS,
  MAX(ArrDelay) AS MAX_ARRIVAL_DELAY,
  ROUND(AVG(ArrDelay), 2) AS AVG_ARRIVAL_DELAY
FROM workspace.aero2.flights
WHERE ArrDelay IS NOT NULL
GROUP BY DATE_FORMAT(FlightDate, 'yyyy-MM-dd')
ORDER BY DATE;

In [0]:
%sql
select * from daily_flight_stats
     

In [0]:
%sql
-- # 4. Show on a bar chart the number of flights per Carrier. Use the full name of the airline (from airlines.csv).

CREATE OR REPLACE TEMP VIEW carrier_flight_counts AS
SELECT
  f.Carrier AS CARRIER_CODE,
  COUNT(*) AS NUM_FLIGHTS
FROM workspace.aero2.flights f
GROUP BY f.Carrier
ORDER BY NUM_FLIGHTS DESC;   

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW carrier_flights_named AS
SELECT
  a.AIRLINE AS AIRLINE_NAME,
  c.NUM_FLIGHTS
FROM carrier_flight_counts c
JOIN workspace.aero2.airlines a
  ON c.CARRIER_CODE = a.IATA_CODE
ORDER BY NUM_FLIGHTS DESC;

In [0]:
%sql
SELECT * FROM carrier_flights_named

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM carrier_flight_counts LIMIT 10;

In [0]:
%sql
-- 5. Show on a pie chart the number of flights per Carrier but this time only show the individual results for the top 10 Carriers
CREATE OR REPLACE TEMP VIEW top10_carriers AS
SELECT
  Carrier AS CARRIER_CODE,
  COUNT(*) AS NUM_FLIGHTS
FROM workspace.aero2.flights
GROUP BY Carrier
ORDER BY NUM_FLIGHTS DESC
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW other_carriers AS
SELECT
  'OTHER' AS CARRIER_CODE,
  COUNT(*) AS NUM_FLIGHTS
FROM workspace.aero2.flights
WHERE Carrier NOT IN (SELECT CARRIER_CODE FROM top10_carriers);

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW carrier_pie_data AS
SELECT * FROM top10_carriers
UNION ALL
SELECT * FROM other_carriers
ORDER BY NUM_FLIGHTS DESC;

In [0]:
%sql
SELECT * FROM carrier_pie_data;

Databricks visualization. Run in Databricks to view.